In [ ]:
# !pip install openai

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")

print(WEAVIATE_URL[:10])
print(WEAVIATE_KEY[:10])
print(OPENAI_API_KEY[:20])
print(OPENAI_URL[:20])

http://loc
root-user-
sk-proj-iuwKF1Q94jnW
https://api.openai.c


## Generate query from prompt

In [2]:
from openai import OpenAI

openai_client = OpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.openai.com/v1",
)

In [3]:
def generate_query_from_promt(prompt):
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            { "role": "system", "content": "Your job is to extract a query from the provided user prompt, the query will then be used to run a query in a vector database." },
            { 
                "role": "user",
                "content": f"Please give me a 2-3 word query that can be used to find relevant info to the following prompt - {prompt}"
            },
        ]
    )
    return response.choices[0].message.content

In [4]:
# Example of how to generate a query from a prompt
generate_query_from_promt("Where do the tallest penguins live?")

'tallest penguins location'

## Connect to Weaviate

In [4]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY,
    "X-OpenAI-BaseURL": OPENAI_URL
  }
)

print(client.is_ready())

True


## Two-step RAG

In [5]:
def two_step_rag(user_prompt):
    # Step 1
    prompt = user_prompt + " Please only use the provided content with this prompt. Don't make things up."
    
    generated_query = generate_query_from_promt(prompt)
    print("=== Generated Query ===")
    print(f"Generated query: {generated_query}")

    # Step 2
    wiki = client.collections.get("Wiki")

    response = wiki.generate.near_text(
        query=generated_query,
        limit=3,
        grouped_task=prompt,
        grouped_properties=["text", "title"]
    )

    # Print results
    print("\n=== Generated Response ===")
    print(response.generative.text)

    print("\n=== Source ===")
    for item in response.objects:
        print(item.properties)

In [8]:
# two_step_rag("What wild animals do we know about?")
two_step_rag("Please provide an explanation at a PhD level. How do airplanes fly?")

=== Generated Query ===
Generated query: "How airplanes fly"

=== Generated Response ===
The principles governing how airplanes fly can be elucidated by examining both the fundamental aerodynamic concepts and the evolutionary context derived from the study of early flight forms. 

At its core, the ability of an airplane to remain airborne hinges on the interaction between the aircraft and the surrounding air—a fluid dynamic process where the design of the aircraft facilitates the generation of lift. Lift is produced when air moves over and under the wings of the aircraft, creating a pressure differential as dictated by Bernoulli's principle and Newton's third law of motion. Essentially, the wing shape (or airfoil design) is crucial because it is constructed to direct airflow in such a manner that the air pressure on top of the wing is lower than that underneath it. This difference in pressure results in lift, allowing the aircraft to ascend into the atmosphere.

Moreover, advancements 

In [9]:
two_step_rag("What are the pros and cons of automation using computer?")

=== Generated Query ===
Generated query: Pros and cons automation computer.

=== Generated Response ===
Based on the provided content, here are the pros and cons of automation using computers:

**Pros:**
1. **Increased Efficiency:** Automation can streamline processes, potentially leading to faster production and operation times.
2. **Innovation in Technology:** With advancements such as faster CPUs and dedicated co-processors, computers facilitate more complex and capable automated systems.
3. **Ease of Software Creation:** Tools like Autopackage simplify the process of creating programs that can be used across various platforms, enhancing accessibility and functionality in automation.

**Cons:**
1. **Decreased Use of Older Technologies:** As automation and modern technologies evolve, older interfaces and methods, like the parallel port, may become obsolete, potentially leading to compatibility issues.
2. **Dependence on Technology:** Increasing reliance on automated systems can lead 

In [10]:
two_step_rag("How do CPUs work?")

=== Generated Query ===
Generated query: How CPUs work

=== Generated Response ===
CPUs (Central Processing Units) work by reading and writing data in RAM (Random Access Memory) through memory addresses. Each byte in RAM has a unique memory address, which the CPU outputs when it needs to access data. The size of these addresses is typically aligned with the word size of the CPU; for example, a 32-bit CPU uses 32-bit addresses. Smaller CPUs, like 8-bit CPUs, may have addresses that are larger than their word size to accommodate longer program lengths.

Within the CPU, execution units perform specific operations or calculations. These execution units can have their own internal control sequence units, registers, and components like sub-ALUs (Arithmetic Logic Units) or FPUs (Floating Point Units). 

Overall, the combined operation of addressing memory and executing calculations enables the CPU to perform tasks effectively.

=== Source ===
{'text': 'When the CPU wants to read or write data

## Close the client

In [11]:
client.close()